In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell.yml")
model = Supervised.build(cfg)

In [2]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
encoder_out = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(utterance.shape)
print(utterance_mask.shape)
print(encoder_out.shape)
print(prev_output.shape)

torch.Size([84, 20])
torch.Size([84, 20])
torch.Size([84, 20, 768])
torch.Size([84, 19])


In [3]:
utterance_mask

tensor([[1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')

In [4]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

torch.Size([1596])

In [7]:
import nlptools.zoo.encoders.transformer
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
from nlptools.zoo.encoders.transformer import TransformerDecoder
embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 3072
dropout = 0
decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
decoder = decoder.to("cuda:0")
output = decoder(prev_output, encoder_out, utterance_mask)
output.shape

torch.Size([84, 19, 30522])

In [8]:
decoder

TransformerDecoder(
  (word_embedding): Embedding(30522, 768, padding_idx=0)
  (position_embedding): Embedding(512, 768)
  (layer_norm): BertLayerNorm()
  (layers): ModuleList(
    (0): TransformerDecoderLayer(
      (self_attn): MultiheadAttention(
        (linear_layers): ModuleList(
          (0): Linear(in_features=768, out_features=768, bias=True)
          (1): Linear(in_features=768, out_features=768, bias=True)
          (2): Linear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0)
        (output_linear): Linear(in_features=768, out_features=768, bias=True)
      )
      (encoder_attn): MultiheadAttention(
        (linear_layers): ModuleList(
          (0): Linear(in_features=768, out_features=768, bias=True)
          (1): Linear(in_features=768, out_features=768, bias=True)
          (2): Linear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0)
        (output_linear): Linear(in_features=768, out_fea